# **Hackmageddon.**
El CSV se centra en incidentes de ciberseguridad, específicamente en ataques de secuestro de cuentas.

## Importación de librerias.

In [1]:
# === Librerías estándar de Python ===
import os

# === Librerías de análisis de datos y visualización ===
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# === Librerías de preprocesamiento de datos ===
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, LabelEncoder
from urllib.parse import urlparse

# === Librerías para aprendizaje supervisado ===
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

# === Librerías para métricas de evaluación ===
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    roc_curve,
    auc,
    precision_score,
    recall_score,
)

# === Librerías para aprendizaje no supervisado ===
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score, silhouette_samples
from scipy.cluster.hierarchy import dendrogram, linkage

# === Librerías para balanceo de datos ===
from imblearn.over_sampling import RandomOverSampler

# === Librerías adicionales ===
from tabulate import tabulate

# === Configuración de colores ===
denexus_colors = ['#1B2D40', '#577596', '#C7262B', '#F85C37', '#FF8250', '#4A4A4A']
denexus_palette = sns.light_palette("#1B2D40", n_colors=100, as_cmap=True)


## Importación e información del DataFrame.

In [2]:
df = pd.read_csv('data/HACKMAGEDDON.csv')
df

FileNotFoundError: [Errno 2] No such file or directory: 'data/HACKMAGEDDON.csv'

In [ ]:
df.describe(include='all')

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.columns

Se nos muestra que tiene un total de 12 columnas que deberemos de examinar para saber cuáles tener en cuenta a la hora de la realización de nuestro estudio y lo q significan cada una de ellas.
1. **Date Reported**: La fecha en que el incidente fue reportado por primera vez a la organización o autoridad competente. Puede diferir de la fecha en que ocurrió el ataque.   
2. **Date Occurred**: La fecha en que ocurrió el incidente o ataque. En algunos casos, puede ser desconocida si el ataque no fue detectado inmediatamente.     
3. **Date Discovered**: La fecha en que se descubrió el incidente. Esta fecha puede ser posterior a la fecha en que ocurrió el evento, especialmente si el ataque pasó desapercibido durante un tiempo.    
4. **Author**: La entidad o persona que reportó o documentó el incidente. Puede ser un investigador, analista de seguridad o un sistema automatizado de detección            
5. **Target**: La entidad o sistema que fue el objetivo del ataque. Puede ser una empresa, un gobierno, un servidor específico, o una persona            
6. **Description**: Una descripción detallada del incidente o ataque. Aquí se proporcionan detalles técnicos o generales sobre lo sucedido.       
7. **Attack**: El tipo de ataque llevado a cabo, por ejemplo, "phishing", "ransomware", "ataque de denegación de servicio (DoS)", etc.             
8. **Target Class**: La clase o tipo de objetivo del ataque. Puede referirse a categorías como "infraestructura crítica", "empresa privada", "usuario final", entre otros.       
9. **Attack Class**: Clasificación del ataque según su naturaleza. Por ejemplo, "intrusión", "exfiltración de datos", "interrupción de servicio", etc.      
10. **Country**: El país de origen del ataque, o donde se encuentra el objetivo del ataque.            
11. **Link**: Un enlace (URL) a una fuente externa que proporciona más información sobre el incidente o ataque.               
12. **Tags**: Etiquetas que ayudan a clasificar el incidente, a menudo utilizadas para categorizar o identificar temas comunes en los datos, como "malware", "APT (Advanced Persistent Threat)", "fraude financiero", etc.               

### **Date Reported.**

Transformaremos la columna **Date Reported** en tres nuevas columnas que representarán el día, mes y año por separado. Esto permitirá realizar un análisis más detallado de los datos basados en fechas específicas.

In [ ]:
df['Date Reported'] = pd.to_datetime(df['Date Reported'])

df['Day'] = df['Date Reported'].dt.day
df['Month'] = df['Date Reported'].dt.month
df['Year'] = df['Date Reported'].dt.year

In [ ]:
df.drop(columns=['Date Reported'], inplace=True)
df.head()

## Limpieza de datos.

### Manejo de filas duplicadas.
En caso de haber filas duplicadas, deberemos de eliminar estas para que no afecten al análisis posterior.

In [ ]:
df.duplicated().sum() # Columnas duplicadas

In [ ]:
df.drop_duplicates(inplace = True)

In [ ]:
df.shape

### Manejo de valores nulos.

In [ ]:
df.isnull().sum()

Se decidió eliminar las columnas **Date Ocurred** y **Date Discovered** debido a la alta proporción de valores `NaN`, que supera el 50%. Esta columna no aportaba información significativa para el análisis y su presencia podría afectar la calidad y el rendimiento del modelo. Mantener columnas con muchos valores nulos complicaría el preprocesamiento de datos y no contribuiría al aprendizaje del modelo.

In [ ]:
df.drop(['Date Occurred', 'Date Discovered'], axis=1, inplace=True)

In [ ]:
df.shape

Se han sustituido los valores nulos en el conjunto de datos para garantizar la integridad y consistencia del análisis. Las columnas con datos numéricos se completaron con su mediana, mientras que las columnas categóricas se rellenaron con su moda, seleccionando el valor más frecuente.

In [ ]:
for column in df.columns:
    if df[column].isnull().sum() > 0:
        if df[column].dtype in ['float64', 'int64']:
            df[column].fillna(df[column].median(), inplace=True)
        else:  # Datos categóricos.s
            df[column].fillna(df[column].mode()[0], inplace=True)

Finalmente, tendríamos nuestro dataframe lleno, sin valores duplicados y nulos.

### Selección de variables categóricas y continuas.
Para realizar correctamente un ánalisis de datos debemos de distinguir entre las variables categóricas, variables que representan diferentes categorías o grupos.

In [ ]:
v_continuas = []
v_categoricas = []
for i in df.columns:
    if df[i].nunique() > 200 or df[i].dtypes in ['float64', 'int64']:
        v_continuas.append(i)
    else:
        v_categoricas.append(i)

print('Variables continuas: {}'.format(', '.join(v_continuas)))
print('Variables categóricas: {}'.format(', '.join(v_categoricas)))

#### Tratamiento de variables categóricas.
Para el posterior modelo de clasificación, necesitamos codificar nuestras variables. Para ello, realizaremos una codificación ordinal que consiste en asignar valores enteros a las categorias basándose en su posición o jerarquía en los datos.

In [ ]:
df[v_categoricas].head(3)

In [ ]:
df[v_categoricas].nunique()

In [ ]:
df[v_categoricas].describe(include='all')

##### **Attack.**

In [ ]:
df['Attack'].unique()

In [ ]:
df['Attack'] = df['Attack'].apply(lambda x: 'Malware' if x.startswith('Malicious') else x)

In [ ]:
attack_types = {
    'Account Hijacking': ['Account Hijacking', 'Account Takeover', 'Account hijacking',
                          'Credential Stuffing', 'Password-spraying', 'Credential stuffing',
                          'Password-Spraying/Credential Stuffing', 'Brute-Force', 'SIM swap', 'Deepfake',
                          'DNS Hijacking', 'Wiretapping'
                        ],
    'Phishing': ['Fake Social Network accounts/groups/pages', 'Fake Social Accounts', 'Fake Websites',
                 'Fake job listings', 'OAuth token hijacking', 'Fake social Network accounts/groups/pages',
                 'Phishing'
                 ],
    'Denial of Service': ['DDoS', 'Server-Side Ad Insertion (SSAI) Hijacking', 'Zoom bombing',
                          'Zoom Bombing', 'Denial of Service'
                         ],
    'Vulnerability Exploitation': ['Vulnerability', 'SQL Injection', 'API Exploit', 'Brute-force',
                                   'Evil cursor', 'SQLi','SSL Stripping', 'Compromised certificate',
                                   'Watering Hole', 'Vulnerability Exploitation', 'HTML smuggling',
                                   'PetitPotam NTLM Relay ', 'Reentrancy attack'
                                  ],
    'Data Breaches': ['Data-Scraping', 'Data Breaches', 'Wind River Systems'],
    'Malvertising': ['Malvertising', 'Malicious Google search ads'],
    'Social Engineering': ['Business Email Compromise', 'Crypto Scam', 'Crypto scam',
                           'Social Engineering', 'Robocalls'
                          ],
    'Targeted Attack': ['Targeted attack', 'Targeted Attack', 'Evil Contract'],
    'DNS Hijacking': ['DNS hijacking', 'DNS Hijacking'],
    'Malware': ['Malware', 'Web Shells', 'Windows Bit locker', 'Malicous Autodesk plugin', 'Web shells'],
    'Defacement': ['Defacement'],
    'Exploitation of Infrastructure': ['51% Attack', 'Bitcoin vanity addresses', 'ATM "Black Box"',
                                       'Jackpotting', 'Search Engine Poisoning', 'SEO Spam',
                                       'Misconfiguration'
                                      ],
    'Spoofing': ['Domain Spoofing']
}


In [ ]:
def categorize_attack(attack):
    for category, attacks in attack_types.items():
        if attack in attacks:
            return category
    return attack

In [ ]:
df['AttackGroup'] = df['Attack'].apply(categorize_attack)

El siguiente código se ha agregado para lograr que las columnas codificadas se agreguen justo después de la columna original, de manera que sea más fácil ver a qué se le ha asignado el valor codificado.

In [ ]:
columns = df.columns.tolist()
index = columns.index('AttackGroup')
columns.insert(index + 1, columns.pop())
df = df[columns]

In [ ]:
encoder = LabelEncoder()
df['AttackEncoded'] = encoder.fit_transform(df['AttackGroup'])

In [ ]:
df.head(2)

##### **Attack Class.**
Debemos revisar los valores únicos en la columna para poder codificarlos adecuadamente. Sin embargo, observamos que hay una categoría que indica más de un tipo de ataque, representada como `'>1'` y `‘CC/CE’`. Por ello, es importante verificar cuántas filas contienen estos valores. Si el número de filas con estos valores es significativo, consideraremos crear columnas adicionales que representen cada tipo de ataque con un 1 o 0, indicando su presencia. En caso contrario, si no hay muchas filas, simplemente eliminaremos estos registros.

In [ ]:
df['Attack Class'].unique()

In [ ]:
df['Attack Class'].value_counts()

Se han eliminado las filas con valores categóricos 'CC/CE' y '>1' para simplificar el análisis y evitar la necesidad de crear variables dummy. Esto permite trabajar con datos más limpios y fáciles de procesar en modelos de aprendizaje automático.

In [ ]:
index_to_drop = df[df['Attack Class'].isin(['CC/CE', '>1'])].index
df.drop(index=index_to_drop, inplace=True)

In [ ]:
encoder = LabelEncoder()
df['AttackClassEncoded'] = encoder.fit_transform(df['Attack Class'])
df.head(2)

##### **Target Class.**
Para agrupar las categorías en la columna **Target Class**, podemos crear un nuevo diccionario que asocie cada categoría original a un grupo más general. Esto ayudará a simplificar el análisis y facilitar el uso de técnicas de aprendizaje automático.

In [ ]:
df['Target Class'].unique()

In [ ]:
target_class_mapping = {
    'Financial Services': ['Finance', 'Fintech', 'Financial and insurance activities','K Financial and insurance activities','V Fintech','Finance and Insurance','Finance and insurance'],
    'Healthcare/ Social Services': ['Human health and social work', 'Health', 'Social', 'Social security','Q Human health and social work activities'],
    'Public Sector/ Government': ['Extraterritorial organizations and bodies','U Activities of extraterritorial organizations and bodies','Public admin and defence', 'Public administration and defence', 'Government','O Public administration and defence, compulsory social security','N Administrative and support service activities','Public admin and defence, social security'],
    'Education': ['Education','P Education'],
    'Manufacturing/ Energy': ['Manufacturing', 'Electricity', 'Electricity gas steam and air conditioning supply','C Manufacturing','D Electricity gas steam and air conditioning supply','Electricity, gas steam, air conditioning'],
    'Information/ Communication': ['M Professional scientific and technical activities','J Information and communication','Information', 'Communication', 'Information and communication'],
    'Arts/ Entertainment': ['Arts entertainment, recreation','Arts entertainment', 'Entertainment', 'Recreation','R Arts entertainment and recreation'],
    'Multiple Industries': ['Multiple Industries', 'Y Multiple Industries', 'X Individual', 'Multiple targets'],
    'Retail Trade': ['Wholesale and retail trade', 'Wholesale and retail','G Wholesale and retail trade'],
    'Transportation': ['H Transportation and storage','Transportation and storage', 'Transport'],
    'Professional Services': ['Administration and support service','M Professional scientific and technical activities', 'Professional, scientific and technical'],
    'Environmental Services': ['E Water supply, sewerage waste management, and remediation activities', 'Water supply, waste mgmt, remediation'],
    'Accommodation and Food Services': ['I Accommodation and food service activities','Accommodation and food service', 'Accommodation and food service activities'],
    'Real Estate': ['L Real estate activities', 'Real estate'],
    'Other': ['Other service activities', 'S Other service activities', 'Unknown', 'Z Unknown']
}


Esta agrupación ofrece múltiples beneficios. Primero, simplifica el análisis al reducir el número de categorías, lo que facilita la interpretación y el manejo de los datos. Además, mejora la eficiencia de los modelos, ya que menos categorías pueden resultar en un mejor rendimiento en el aprendizaje automático. También facilita las visualizaciones, permitiendo crear gráficos más claros y comprensibles, lo que mejora la comunicación de los resultados.

La agrupación contribuye a la reducción del ruido al eliminar categorías poco relevantes, lo que mejora la calidad del análisis y facilita la identificación de tendencias significativas. Asimismo, ayuda en la toma de decisiones al proporcionar un análisis más claro y simplificado, lo que permite a los responsables tomar decisiones informadas basadas en datos. Finalmente, esta agrupación prepara el camino para el aprendizaje supervisado, optimizando el proceso de modelado.




In [ ]:
def assign_target(Target_Class):
    for continent, countries in target_class_mapping.items():
        if Target_Class in countries:
            return continent
    return Target_Class

In [ ]:
df['TargetClassGroup'] = df['Target Class'].map(assign_target)

In [ ]:
encoder = LabelEncoder()
df['TargetClassEncoded'] = encoder.fit_transform(df['TargetClassGroup'])
df.head(2)

##### **Countries.**
Para realizar una evaluación más efectiva de los países en nuestros datos, es conveniente agruparlos por continentes. Esto es especialmente relevante porque en otros archivos CSV hay registros de ataques provenientes de diferentes países, distribuidos en distintos continentes.

Para facilitar este análisis, procederemos a crear seis nuevas columnas en nuestro conjunto de datos, cada una representando un continente específico. Estas columnas indicarán con un valor de 1 si el país pertenece al continente correspondiente y 0 si no es así, es decir, mediante una asignación de valores binarios.

In [ ]:
df['Country'].unique()

In [ ]:
continent_mapping = {
    'North America': ['US', 'CA', 'MX', 'PR', 'JM', 'CW', 'BB', 'CR', 'DO', 'SV', 'KY', 'NI', 'AG','US\r\n CA\r\n MX','US\r\n UK','US\r\n CA','US\r\n HK','INT','CA\r\n JP','US\r\n AU','US\r\n KR','US/UK','US, CA','US, DE, AT','US/CA','US, IL','UK/US','US\r\nCA','US\r\nIL','Multiple targets'],
    'Europe': ['AT', 'SL', 'DE', 'UA', 'CZ', 'GR', 'TR', 'MT', 'IT', 'UK', 'NL', 'FR', 'ES', 'HR', 'DK', 'RU', 'CH', 'PT', 'EE', 'PL', 'SK', 'NO', 'MK', 'CY', 'VA', 'BE', 'HU', 'SE', 'FI', 'GI', 'LI', 'RO', 'MD', 'AL','EU','US\r\n UK', 'RS','ES\r\n PT','INT','IE''IL\r\n UK','DE\r\n NL','US/UK','BY','SI','US, DE, AT','LT','NL/UK','LU','UK/US','AU\r\nDE','AU\r\nTR','IT\r\nFR','IE','Multiple targets'],
    'Asia': ['NP\r\n AF', 'KR', 'BH', 'KW', 'OM', 'JP', 'IL', 'HK', 'IN', 'MY', 'IR', 'PS', 'PK', 'SG', 'LB', 'CN', 'VN', 'MN', 'AM', 'GE', 'SA', 'LK', 'SY', 'NP', 'AZ', 'ID', 'TW', 'TH', 'UAE', 'PH', 'AF', 'KZ', 'BD', 'QA', 'KH', 'LA','US\r\n HK','SA\r\n AE','INT','CA\r\n JP','IL\r\n UK','US\r\n KR','TW\r\n PH','KR\r\n TW','KW\r\n SA','IN\r\n HK','MM','US, IL','JO','TW\r\nKR','US\r\nIL','IN\r\nAF','LY\r\nSY','Multiple targets'],
    'Australia': ['AU', 'NZ', 'FJ', 'PG','INT','US\r\n AU','AU\r\nDE','AU\r\nTR','Multiple targets'],
    'South America': ['BR', 'PE', 'AR', 'CL', 'CO', 'VE', 'EC','INT','Multiple targets'],
    'Africa': ['TZ','ZA', 'RW', 'SC', 'MA', 'LY', 'DZ', 'AO', 'UG', 'CV', 'EG', 'CI', 'TG', 'TN','INT','KE','IN\r\nAF','CF','LY\r\nSY','Multiple targets']
}

In [ ]:
def assign_continent(country):
    for continent, countries in continent_mapping.items():
        if country in countries:
            return continent
    return None  # Si el país no está en la lista, retornar None

In [ ]:
df['Continent'] = df['Country'].map(assign_continent)

In [ ]:
countries_with_no_continent = df[df['Continent'].isnull()]['Country'].unique()

# Mostrar la lista de países con 'continent' como None
print("Lista de países con 'Continent' como None:")
print(countries_with_no_continent)

In [ ]:
df[df['Country'].isin(countries_with_no_continent)].shape[0]

Dado que se trata de un porcentaje significativo de datos, optaremos por reemplazar estos valores utilizando la mediana de la columna correspondiente. La **mediana** es una medida de estimación eficaz para esta sustitución, ya que no se ve influenciada por valores atípicos, a diferencia de la media. En nuestro caso al ser una variable de tipo object, utilizaremos la moda.

In [ ]:
df['Continent'].fillna(df['Continent'].mode()[0], inplace=True)

In [ ]:
df_continentes = pd.get_dummies(df['Continent'])
df = pd.concat([df, df_continentes], axis=1)

In [ ]:
df.head(2)

In [ ]:
df.drop(['Country', 'Continent'], axis = 1, inplace = True)

#### Tratamiento de las variables continuas.
Para el modelo de clasificación debemos de pasar todas las columnas a tipo int. Por ello, debemos de ver si realizar como en el caso anterior una codificación ordinal o agruparlos.

In [ ]:
df[v_continuas].head(2)

In [ ]:
v_categoricas_cont = list(df[v_continuas].dtypes[df[v_continuas].dtypes == 'object'].index)

In [ ]:
df[v_categoricas_cont].nunique()

##### **Author.**
En los datos analizados, existen numerosos tipos de actores involucrados, lo que puede dificultar la clasificación y el análisis detallado. Para simplificar y garantizar una mayor claridad, hemos decidido agruparlos en dos categorías principales: **conocidos** y **desconocidos**. Esta estrategia permite un enfoque más estructurado y facilita la interpretación de los resultados.

In [ ]:
df['AuthorKnown'] = df['Author'].apply(lambda x: 0 if x == '?' else 1)

##### **Description.**
La columna contiene valores que en su mayoría son únicos, lo que sugiere que actúa más como un identificador que como una característica con patrones generales útiles para el modelo. Transformarla directamente podría generar un exceso de categorías o introducir relaciones artificiales entre los valores, lo que podría afectar negativamente el rendimiento del modelo. Por este motivo, es mejor no transformar esta columna y evaluar su relevancia antes de incluirla en el análisis.

##### **Target.**
La columna contiene valores que en su mayoría son únicos, lo que sugiere que actúa más como un identificador que como una característica con patrones generales útiles para el modelo. Transformarla directamente podría generar un exceso de categorías o introducir relaciones artificiales entre los valores, lo que podría afectar negativamente el rendimiento del modelo. Por este motivo, es mejor no transformar esta columna y evaluar su relevancia antes de incluirla en el análisis.

In [ ]:
df['Target'].unique()

In [ ]:
df['Target'].nunique()

##### **Link**.
Esta columna será útil para definir las relaciones entre diferentes CSVs. Extraeremos el nombre del dominio de la URL para simplificar el análisis y la comparación entre los datos.

In [ ]:
def obtener_dominio(url):
    try:
        return urlparse(url).netloc
    except:
        return None

In [ ]:
df['Link'] = df['Link'].apply(obtener_dominio)
df['Link'] = df['Link'].str.replace('www.', '', regex=False)

In [ ]:
df.head()

In [ ]:
df.to_csv('data/HACKMAGEDDON_cleaned.csv', index=False)

## Modelos de clasificación.
Para la realización de los modelos de clasificación, deberemos de utilizar las columnas numéricas. Estas columnas se utilizarán como características para entrenar los modelos, permitiendo que estos aprendan patrones en los datos.

Este proceso tiene como objetivo comprobar la limpieza de los datos y garantizar que la información sea adecuada para el análisis posterior.

In [ ]:
df_1 = df.drop(list(df.select_dtypes(include=['object']).columns), axis=1)
df_1.head()

### Modelos de Clasificación Supervisado.
Hemos seleccionado **Author Known** como la variable objetivo porque representa si se supo quién fue el ataquante o no, siendo esta una variable binaria.

In [ ]:
results = pd.DataFrame(columns=['Model', 'Tipo', 'Accuracy', 'Recall'])
results['Tipo'] = ['Binary']*3

#### Matriz de correlación.

In [ ]:
correlation_matrix = df_1.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap=denexus_palette, fmt=".2f")
plt.title('Correlation Matrix between Continuous Variables')
plt.show()

#### División en Conjuntos de Entrenamiento y Prueba.

In [ ]:
X = df_1.drop(['AuthorKnown'], axis=1)
y = df_1['AuthorKnown']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state=42)

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

#### Manejo de variables desbalanceadas.
Dado el desbalance de clases en el conjunto de entrenamiento, vamos a aplicar **Random Over-Sampling (ROS)** para equilibrar las clases minoritarias. Esta técnica aumenta la representación de las clases menos frecuentes duplicando aleatoriamente sus instancias, lo que permite que el modelo aprenda de manera más equitativa.

In [ ]:
ros = RandomOverSampler(random_state=42)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

#### Entrenamiento y Evaluación de Modelos.
En este apartado, entrenaremos modelos de aprendizaje supervisado para predecir el si el autor fue conocido o no.

##### **Random Forest.**
**Random Forest** es un algoritmo de aprendizaje automático basado en la creación de múltiples árboles de decisión, cuyos resultados se combinan para mejorar la precisión y evitar el sobreajuste.

In [ ]:
rf_model = RandomForestClassifier(random_state=42,)
rf_model.fit(X_train_ros, y_train_ros)

In [ ]:
feature_importances = rf_model.feature_importances_

importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(importance_df['Feature'], importance_df['Importance'], align='center', color= denexus_colors[2])
plt.gca().invert_yaxis()
plt.xlabel('Importance Score')
plt.ylabel('Features')
plt.title('Feature Importance - Random Forest')
plt.tight_layout()
# Guardar el gráfico en la carpeta 'plots'.
output_dir = "plots"
if os.path.exists(output_dir) and os.path.isdir(output_dir):
    output_path = os.path.join(output_dir, "feature-importance_random-forest.png")
    plt.savefig(output_path)
plt.show()

In [ ]:
cv_scores = cross_val_score(rf_model, X_train_ros, y_train_ros, cv=5, scoring='accuracy')
cv_scores

In [ ]:
print("Train set score (Accuracy) =", rf_model.score(X_train_ros, y_train_ros))
print("Test set score (Accuracy) =", rf_model.score(X_test, y_test))

conf_mat = confusion_matrix(y_test, rf_model.predict(X_test))

num_classes = conf_mat.shape[0]

print(tabulate(
    conf_mat,
    headers=[f'Pred Class {i}' for i in range(num_classes)],
    showindex=[f'Real Class {i}' for i in range(num_classes)],
    tablefmt='fancy_grid'
))

print("\nClassification Report:")
print(classification_report(y_test, rf_model.predict(X_test)))

In [ ]:
class_labels = [f'Class {i}' for i in range(conf_mat.shape[0])]

plt.figure(figsize=(8, 6))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap=denexus_palette,
            xticklabels=class_labels, yticklabels=class_labels, cbar=True)
plt.title('Confusion Matrix - Random Forest Classification')
plt.xlabel('Predicted Class')
plt.ylabel('Actual Class')
plt.tight_layout()
# Guardar el gráfico en la carpeta 'plots'.
output_dir = "plots"
if os.path.exists(output_dir) and os.path.isdir(output_dir):
    output_path = os.path.join(output_dir, "confusion-matrix_random-forest.png")
    plt.savefig(output_path)
plt.show()

##### **KNeighbors.**

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_ros, y_train_ros)

In [ ]:
cv_scores = cross_val_score(knn_model, X_train_ros, y_train_ros, cv=5, scoring='accuracy')
cv_scores

In [ ]:
print("Train set score (Accuracy) =", knn_model.score(X_train_ros, y_train_ros))
print("Test set score (Accuracy) =", knn_model.score(X_test, y_test))

conf_mat = confusion_matrix(y_test, knn_model.predict(X_test))

num_classes = conf_mat.shape[0]

print(tabulate(
    conf_mat,
    headers=[f'Pred Class {i}' for i in range(num_classes)],
    showindex=[f'Real Class {i}' for i in range(num_classes)],
    tablefmt='fancy_grid'
))

print("\nClassification Report:")
print(classification_report(y_test, knn_model.predict(X_test)))

In [ ]:
class_labels = [f'Class {i}' for i in range(conf_mat.shape[0])]

plt.figure(figsize=(8, 6))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap=denexus_palette,
            xticklabels=class_labels, yticklabels=class_labels, cbar=True)
plt.title('Confusion Matrix - KNN Classification')
plt.xlabel('Predicted Class')
plt.ylabel('Actual Class')
plt.tight_layout()
# Guardar el gráfico en la carpeta 'plots'.
output_dir = "plots"
if os.path.exists(output_dir) and os.path.isdir(output_dir):
    output_path = os.path.join(output_dir, "confusion-matrix_knn.png")
    plt.savefig(output_path)
plt.show()

##### **XGBoost Model.**

In [ ]:
xgboost_model = XGBClassifier(eval_metric='logloss')

In [ ]:
xgboost_model.fit(X_train_ros, y_train_ros)

In [ ]:
Y_pred_xgb = xgboost_model.predict(X_train_ros)
print("XGBoost Classification Report:\n")
print("Test set score (Accuracy) =", xgboost_model.score(X_test, y_test))

conf_mat = confusion_matrix(y_test, xgboost_model.predict(X_test))
num_classes = conf_mat.shape[0]

print(tabulate(
    conf_mat,
    headers=[f'Pred Class {i}' for i in range(num_classes)],
    showindex=[f'Real Class {i}' for i in range(num_classes)],
    tablefmt='fancy_grid'
))

print("\nClassification Report:")
print(classification_report(y_test, xgboost_model.predict(X_test)))

In [ ]:
class_labels = [f'Class {i}' for i in range(conf_mat.shape[0])]

plt.figure(figsize=(8, 6))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap=denexus_palette,
            xticklabels=class_labels, yticklabels=class_labels, cbar=True)
plt.title('Confusion Matrix - XGBoost Classification')
plt.xlabel('Predicted Class')
plt.ylabel('Actual Class')
plt.tight_layout()
# Guardar el gráfico en la carpeta 'plots'.
output_dir = "plots"
if os.path.exists(output_dir) and os.path.isdir(output_dir):
    output_path = os.path.join(output_dir, "confusion-matrix_xgboost.png")
    plt.savefig(output_path)
plt.show()

#### Curva ROC.

In [ ]:
y_prob_rf = rf_model.predict_proba(X_test)[:, 1]
y_prob_knn = knn_model.predict_proba(X_test)[:, 1]
y_prob_xgb = xgboost_model.predict_proba(X_test)[:, 1]

In [ ]:
# Calcular FPR, TPR y el AUC.
## Random Forest.
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_prob_rf)
roc_auc_rf = auc(fpr_rf, tpr_rf)

## Decision Tree.
fpr_knn, tpr_knn, _ = roc_curve(y_test, y_prob_knn)
roc_auc_knn = auc(fpr_knn, tpr_knn)

## XGBoost.
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, y_prob_xgb)
roc_auc_xgb = auc(fpr_xgb, tpr_xgb)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(20, 6))

## Subplot para Random Forest.
axs[0].plot(fpr_rf, tpr_rf, label=f'ROC Curve RF (AUC = {roc_auc_rf:.2f})', color=denexus_colors[1])
axs[0].plot([0, 1], [0, 1], color=denexus_colors[2], linestyle='--', label='Random Guess')
axs[0].set_title('ROC Curve - Random Forest')
axs[0].set_xlabel('False Positive Rate (FPR)')
axs[0].set_ylabel('True Positive Rate (TPR)')
axs[0].legend(loc='lower right')
axs[0].grid(True)

## Subplot para Decision Tree.
axs[1].plot(fpr_knn, tpr_knn, label=f'ROC Curve KNN (AUC = {roc_auc_knn:.2f})', color=denexus_colors[1])
axs[1].plot([0, 1], [0, 1], color=denexus_colors[2], linestyle='--', label='Random Guess')
axs[1].set_title('ROC Curve - KNN')
axs[1].set_xlabel('False Positive Rate (FPR)')
axs[1].set_ylabel('True Positive Rate (TPR)')
axs[1].legend(loc='lower right')
axs[1].grid(True)

## Subplot para XGBoost.
axs[2].plot(fpr_xgb, tpr_xgb, label=f'ROC Curve XGB (AUC = {roc_auc_xgb:.2f})', color=denexus_colors[1])
axs[2].plot([0, 1], [0, 1], color=denexus_colors[2], linestyle='--', label='Random Guess')
axs[2].set_title('ROC Curve - XGBoost')
axs[2].set_xlabel('False Positive Rate (FPR)')
axs[2].set_ylabel('True Positive Rate (TPR)')
axs[2].legend(loc='lower right')
axs[2].grid(True)

plt.tight_layout()

# Guardar el gráfico en la carpeta 'plots'.
output_dir = "plots"
if os.path.exists(output_dir) and os.path.isdir(output_dir):
    output_path = os.path.join(output_dir, "ROC-curves.png")
    plt.savefig(output_path)

plt.show()

#### Resultados.

In [ ]:
rf_pred = rf_model.predict(X_test)
knn_pred = knn_model.predict(X_test)
xgb_pred = xgboost_model.predict(X_test)

In [ ]:
rf_precision = precision_score(y_test, rf_pred, average='weighted')
rf_recall = recall_score(y_test, rf_pred, average='weighted')

In [ ]:
knn_precision = precision_score(y_test, knn_pred, average='weighted')
knn_recall = recall_score(y_test, knn_pred, average='weighted')

In [ ]:
xgb_precision = precision_score(y_test, xgb_pred, average='weighted')
xgb_recall = recall_score(y_test, xgb_pred, average='weighted')

In [ ]:
results['Model'] = ['RandomForest', 'KNN', 'XGBoost']
results['Accuracy'] = [rf_precision, knn_precision, xgb_precision]
results['Recall'] = [rf_recall, knn_recall, xgb_recall]

In [ ]:
results

### Modelos de Clasificación No Supervisados.

Para la realización de los modelos de clasificación, deberemos utilizar las columnas numéricas. Estas columnas se utilizarán como características para entrenar los modelos, permitiendo que estos aprendan patrones en los datos.


In [ ]:
df_1 = df.drop(list(df.select_dtypes(include=['object']).columns), axis=1)
df_1.head()

#### Preprocesamiento de Datos


##### Aplicar PCA para agrupar variables en Componentes Principales

In [ ]:
# === Escalado de los datos numéricos de df_1 ===
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_1)

# === Número de componentes principales
pca_optimal = PCA(n_components=8)
pca_result_optimal = pca_optimal.fit_transform(df_scaled)

##### Número Óptimo de Componentes Principales (≥80% Varianza)

In [ ]:
# PCA para calcular todas las componentes
pca_full = PCA()
pca_full.fit(df_scaled)

# Varianza explicada por cada componente y acumulada
explained_variance = pca_full.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

# Determinamos el número de componentes que explican al menos el 75% de la varianza
optimal_components = np.argmax(cumulative_variance >= 0.75) + 1

print(f"Número óptimo de componentes para el 75% de la varianza: {optimal_components}")

# Gráfica de la varianza acumulada
plt.figure(figsize=(8, 6))

plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance,
         marker='o', linestyle='--', color=denexus_colors[1])
plt.axhline(y=0.75, color=denexus_colors[2], linestyle='-')

plt.title('Varianza Acumulada por Número de Componentes', color=denexus_colors[0])
plt.xlabel('Número de Componentes Principales', color=denexus_colors[0])
plt.ylabel('Proporción de Varianza Acumulada', color=denexus_colors[0])

plt.grid(color=denexus_colors[5], linestyle='--', alpha=0.5)
plt.show()


#### Aplicación de Métodos de Clustering

##### Clustering: K-Means

In [ ]:
# =====Utilizar 4 Clusters.======


# Número de clusters para K-Means
n_clusters = 4  # Ahora usamos 4 clusters

# Generamos pares de PCAs
pairs = [(i, j) for i in range(8) for j in range(i + 1, 8)]  # Todas las combinaciones de pares de PCAs

# Creamos un DataFrame con las 8 componentes principales
df_pca_8 = pd.DataFrame(pca_result_optimal, columns=[f'PC{i+1}' for i in range(8)])

# Definimos colores personalizados para los clusters
colors = denexus_colors

# Visualizamos K-Means en cada par de PCAs
plt.figure(figsize=(20, 15))  # Ajustamos el tamaño del grid de gráficas
rows = 4  # Filas del grid
cols = 7  # Columnas del grid (en total habrá 28 gráficas)

for k, (pc_x, pc_y) in enumerate(pairs):
    # Creamos un modelo K-Means para los dos PCAs seleccionados
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)

    # Extraemos las dos componentes principales actuales
    X_pair = df_pca_8[[f'PC{pc_x + 1}', f'PC{pc_y + 1}']].values

    # Ajustamos K-Means y predecimos los clusters
    kmeans_labels = kmeans.fit_predict(X_pair)

    # Graficamos el resultado
    plt.subplot(rows, cols, k + 1)
    for cluster in range(n_clusters):
        plt.scatter(X_pair[kmeans_labels == cluster, 0],
                    X_pair[kmeans_labels == cluster, 1],
                    color=colors[cluster], label=f'Cluster {cluster}', alpha=0.5)
    plt.title(f"K-Means (4 clusters): PC{pc_x + 1} vs PC{pc_y + 1}")
    plt.xlabel(f"PC{pc_x + 1}")
    plt.ylabel(f"PC{pc_y + 1}")
    plt.grid()

plt.tight_layout()
plt.show()

In [ ]:

#======= Visualizar cada pareja de Componentes Principales.=======

# PCA con 8 componentes principales
pca_optimal = PCA(n_components=8)
pca_result_optimal = pca_optimal.fit_transform(df_scaled)


# Varianza explicada por cada componente principal
explained_variance = pca_optimal.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

# Mostramos la varianza explicada y acumulada
print(f"Varianza explicada por cada componente principal: {explained_variance}")
print(f"Varianza total explicada (8 componentes): {np.sum(explained_variance):.2f}")

# Cambiamos la cuadrícula a 7 filas y 4 columnas (28 subplots en total)
plt.figure(figsize=(20, 30))  # Ajustamos el tamaño de la figura

rows = 7  # Suficiente para 28 subplots
cols = 4

for k, (pc_x, pc_y) in enumerate(pairs):
    plt.subplot(rows, cols, k + 1)  # Suficientes subplots para 28 gráficos
    color = denexus_colors[k % len(denexus_colors)]
    plt.scatter(df_pca_8[f'PC{pc_x+1}'], df_pca_8[f'PC{pc_y+1}'], alpha=0.5, color=color)
    plt.title(f"PC{pc_x+1} vs PC{pc_y+1}")
    plt.xlabel(f"PC{pc_x+1}")
    plt.ylabel(f"PC{pc_y+1}")
    plt.grid()

plt.tight_layout()
plt.show()

##### Clustering: DBSCAN

In [ ]:

# Parámetros de DBSCAN (puedes ajustarlos según el dataset)
eps = 0.5  # Distancia máxima para considerar puntos vecinos
min_samples = 5  # Número mínimo de puntos para formar un cluster

# Generamos pares de PCAs
pairs = [(i, j) for i in range(8) for j in range(i + 1, 8)]  # Todas las combinaciones de pares de PCAs

# Definimos colores personalizados (incluyendo uno para ruido)
denexus_colors_extended = denexus_colors + ['#808080'] # Gris para ruido

# Visualizamos DBSCAN en cada par de PCAs
plt.figure(figsize=(20, 15))  # Ajustamos el tamaño del grid de gráficas
rows = 4  # Filas del grid
cols = 7  # Columnas del grid (en total habrá 28 gráficas)

for k, (pc_x, pc_y) in enumerate(pairs):
    # Creamos un modelo DBSCAN para los dos PCAs seleccionados
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)

    # Extraemos las dos componentes principales actuales
    X_pair = df_pca_8[[f'PC{pc_x + 1}', f'PC{pc_y + 1}']].values

    # Ajustamos DBSCAN y obtenemos las etiquetas de los clusters
    dbscan_labels = dbscan.fit_predict(X_pair)
    unique_labels = set(dbscan_labels)  # Incluye clusters y ruido (-1)

    # Graficamos el resultado
    plt.subplot(rows, cols, k + 1)
    for cluster in unique_labels:
        if cluster == -1:
            # Puntos etiquetados como ruido
            cluster_color = 'gray'
        else:
            # Puntos de clusters válidos
            cluster_color = colors[cluster % len(colors)]  # Reutilizamos colores si hay más de 4 clusters
        plt.scatter(X_pair[dbscan_labels == cluster, 0],
                    X_pair[dbscan_labels == cluster, 1],
                    color=cluster_color, label=f'Cluster {cluster}', alpha=0.5)
    plt.title(f"DBSCAN: PC{pc_x + 1} vs PC{pc_y + 1}")
    plt.xlabel(f"PC{pc_x + 1}")
    plt.ylabel(f"PC{pc_y + 1}")
    plt.grid()

plt.tight_layout()
plt.show()


#### Métodos de Validación


##### Elbow Method

In [ ]:
# === Método del Codo (Elbow Method) ===
inertia = []
range_n_clusters = range(1, 11)  # Probaremos entre 1 y 10 clusters

for n_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(df_pca_8)
    inertia.append(kmeans.inertia_)

# Gráfica del Método del Codo
plt.plot(range_n_clusters, inertia, marker='o', linestyle='--', color=denexus_colors[1])
plt.title('Método del Codo (Elbow Method)', color=denexus_colors[0])
plt.xlabel('Número de Clusters', color=denexus_colors[0])
plt.ylabel('Inercia (Inertia)', color=denexus_colors[0])
plt.grid(color=denexus_colors[5], linestyle='--', alpha=0.5)  # Gris oscuro para la cuadrícula

plt.show()

##### Silhouette Method

In [ ]:
silhouette_avg = []
range_n_clusters = range(2, 11)  # Silhouette requiere al menos 2 clusters

for n_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(df_pca_8)
    silhouette_avg.append(silhouette_score(df_pca_8, cluster_labels))

# Gráfica del Método de la Silueta
plt.plot(range_n_clusters, silhouette_avg, marker='o', linestyle='--', color=denexus_colors[4])
plt.title('Método de la Silueta (Silhouette Method)', color=denexus_colors[0])
plt.xlabel('Número de Clusters', color=denexus_colors[1])
plt.ylabel('Coeficiente de Silueta', color=denexus_colors[1])
plt.grid(color=denexus_colors[5], linestyle='--', alpha=0.5)  # Gris oscuro para la cuadrícula
plt.show()

# Selección del número óptimo de clusters según los métodos
optimal_k_elbow = inertia.index(min(inertia)) + 1  # Según el Codo
optimal_k_silhouette = silhouette_avg.index(max(silhouette_avg)) + 2  # Según la Silueta

print(f"Número óptimo de clusters (Elbow Method): {optimal_k_elbow}")
print(f"Número óptimo de clusters (Silhouette Method): {optimal_k_silhouette}")



## Visualización de las gráficas.

### Distribución de Author, Attack y Target Class.

In [ ]:
import matplotlib.pyplot as plt

# Configuración de colores.
cmap = plt.get_cmap('tab20')

# Crear la figura y los subplots.
fig, axs = plt.subplots(1, 3, figsize=(50, 25))

# Gráfico para 'Author'.
freq_author = df['Author'].value_counts().head(10)
axs[0].pie(
    freq_author.values,
    labels=freq_author.index,
    autopct='%1.1f%%',
    startangle=140,
    colors=denexus_colors[:len(freq_author)],
    textprops={'fontsize': 20},
    wedgeprops={'edgecolor': 'white'}
)
axs[0].set_title("Distribution of 'Author'", fontsize=16)

# Gráfico para 'Attack'.
freq_attack = df['Attack'].value_counts().head(10)
axs[1].pie(
    freq_attack.values,
    labels=freq_attack.index,
    autopct='%1.1f%%',
    startangle=140,
    colors=denexus_colors[:len(freq_attack)],
    textprops={'fontsize': 20},
    wedgeprops={'edgecolor': 'white'}
)
axs[1].set_title("Distribution of 'Attack'", fontsize=16)

# Gráfico para 'Target Class'.
freq_target = df['Target Class'].value_counts().head(10)
axs[2].pie(
    freq_target.values,
    labels=freq_target.index,
    autopct='%1.1f%%',
    startangle=140,
    colors=denexus_colors[:len(freq_target)],
    textprops={'fontsize': 20},
    wedgeprops={'edgecolor': 'white'}
)
axs[2].set_title("Distribution of 'Target Class'", fontsize=16)
plt.tight_layout()
# Guardar el gráfico en la carpeta 'plots'.
output_dir = "plots"
if os.path.exists(output_dir) and os.path.isdir(output_dir):
    output_path = os.path.join(output_dir, "distributions.png")
    plt.savefig(output_path)
plt.show()

### Top 10 valores de Author, Attack y Target Class.

In [ ]:
# Definir el tamaño de la figura para los gráficos
plt.figure(figsize=(10, 6))

# Lista de columnas a graficar
columns_to_plot = ['Attack', 'Target Class', 'Attack Class']

# Crear gráficos de barras para las columnas categóricas
for col in columns_to_plot:
    plt.figure(figsize=(10, 6))
    freq = df[col].value_counts().head(10)  # Top 10 categorías
    sns.barplot(x=freq.index, y=freq.values, palette=denexus_colors)
    plt.title(f"Top 10 values in '{col}'", fontsize=16)
    plt.xlabel(col, fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.xticks(rotation=45)
    plt.tight_layout()
    # Guardar el gráfico en la carpeta 'plots'.
    output_dir = "plots"
    if os.path.exists(output_dir) and os.path.isdir(output_dir):
        output_path = os.path.join(output_dir, f"{col}-distribution.png")
        plt.savefig(output_path)
    plt.show()

### Tendencias a lo largo del tiempo.

In [ ]:
time_columns = ['Day', 'Month']
for col in time_columns:
    plt.figure(figsize=(10, 6))
    freq = df[col].value_counts().sort_index()  # Ordenar por fecha o año
    sns.lineplot(x=freq.index, y=freq.values, marker='o', color=denexus_colors[1])
    plt.title(f"Trends over time in '{col}'", fontsize=16)
    plt.xlabel(col, fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.tight_layout()
    # Guardar el gráfico en la carpeta 'plots'.
    output_dir = "plots"
    if os.path.exists(output_dir) and os.path.isdir(output_dir):
        output_path = os.path.join(output_dir, f"{col}-trends.png")
        plt.savefig(output_path)
    plt.show()

### Distribución de ataques por región.

In [ ]:
regions = ['Africa', 'Asia', 'Europe', 'North America', 'Australia', 'South America']
df_regions = df[regions].sum()
df_regions = df_regions.sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=df_regions.index, y=df_regions.values, palette=denexus_colors)
plt.title("Distribution of Attacks by Region", fontsize=16)
plt.xlabel('Region', fontsize=12)
plt.ylabel('Total Attacks', fontsize=12)
plt.tight_layout()
# Guardar el gráfico en la carpeta 'plots'.
output_dir = "plots"
if os.path.exists(output_dir) and os.path.isdir(output_dir):
    output_path = os.path.join(output_dir, "attacks-by-region.png")
    plt.savefig(output_path)
plt.show()


In [ ]:
columns_to_analyze = ['Author', 'Attack', 'Target Class', 'Link']

for col in columns_to_analyze:
    plt.figure(figsize=(10, 6))
    freq = df[col].value_counts().head(20)
    sns.barplot(x=freq.values, y=freq.index, palette=denexus_colors)

    # Títulos y etiquetas
    plt.title(f"Frequency Distribution of '{col}'", fontsize=16)
    plt.xlabel("Frequency", fontsize=14)
    plt.ylabel(col, fontsize=14)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.tight_layout()
    # Guardar el gráfico en la carpeta 'plots'.
    output_dir = "plots"
    if os.path.exists(output_dir) and os.path.isdir(output_dir):
        output_path = os.path.join(output_dir, f"{col}-frequency-distribution.png")
        plt.savefig(output_path)
    plt.show()
